In [107]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

## Use requests pkg to load in raw bytes of HTML

### .decode() changes bytes to str

In [108]:
resp = requests.get('https://www.basketball-reference.com/leagues/NBA_2023_per_game.html')
html_data = resp.content.decode()

soup = BeautifulSoup(html_data)


### find_all() selects HTML lines based on tags

In [109]:

# [1:] in next line to remove 'Rk' attribute that is arbitrary depending on sorting rule
attribute_headers = [header.getText() for header in soup.find_all('tr',limit=1)[0].find_all('th')][1:]

rows = soup.find_all('tbody',limit=1)[0].find_all('tr',class_='full_table')
player_stats = np.array([[attr.getText() for attr in row.find_all('td') ] for row in rows])


In [110]:

df_NBA_Player_Stats = pd.DataFrame(player_stats, columns=attribute_headers)
df_NBA_Player_Stats = df_NBA_Player_Stats.apply(pd.to_numeric, errors='ignore')
df_NBA_Player_Stats.head()


,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Precious Achiuwa,C,23,TOR,41,11,22.6,3.7,7.8,0.477,...,0.689,2.0,4.3,6.3,1.0,0.6,0.6,1.2,2.0,9.7
1,Steven Adams,C,29,MEM,42,42,27.0,3.7,6.3,0.597,...,0.364,5.1,6.5,11.5,2.3,0.9,1.1,1.9,2.3,8.6
2,Bam Adebayo,C,25,MIA,60,60,35.0,8.4,15.6,0.540,...,0.806,2.6,7.1,9.7,3.3,1.2,0.8,2.5,2.8,21.3
3,Ochai Agbaji,SG,22,UTA,42,5,16.4,2.0,4.2,0.461,...,0.741,0.6,1.1,1.8,0.6,0.2,0.1,0.4,1.5,5.4
4,Santi Aldama,PF,22,MEM,60,18,21.9,3.3,7.0,0.472,...,0.718,1.0,3.6,4.5,1.2,0.7,0.7,0.7,1.9,9.2


In [111]:
#For salary data
resp = requests.get('https://hoopshype.com/salaries/players/')
html_data = resp.content.decode()

soup = BeautifulSoup(html_data)

In [112]:
attribute_headers = [header.getText().strip() for header in soup.find_all('tr',limit=1)[0].find_all('td', limit=3)][1:]
attribute_headers[1] += ' Salary'

In [113]:

rows = soup.find_all('tbody',limit=1)[0].find_all('tr')
rows[0].find_all('td')[1:3][1].getText().strip('')
player_salaries = np.array([[attr.getText().strip('$\t\n').replace(',','') for attr in row.find_all('td')[1:3] ] for row in rows])

In [114]:
df_salaries = pd.DataFrame(player_salaries, columns=attribute_headers)
df_salaries['2022/23 Salary'] = df_salaries['2022/23 Salary'].apply(int)
df_salaries.head()

,Player,2022/23 Salary
0,Stephen Curry,48070014
1,John Wall,47345760
2,Russell Westbrook,47080179
3,LeBron James,44474988
4,Kevin Durant,44119845


In [115]:
x = pd.merge(df_NBA_Player_Stats,df_salaries, on='Player')
x

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,2022/23 Salary
0,Precious Achiuwa,C,23,TOR,41,11,22.6,3.7,7.8,0.477,...,2.0,4.3,6.3,1.0,0.6,0.6,1.2,2.0,9.7,2840160
1,Steven Adams,C,29,MEM,42,42,27.0,3.7,6.3,0.597,...,5.1,6.5,11.5,2.3,0.9,1.1,1.9,2.3,8.6,17926829
2,Bam Adebayo,C,25,MIA,60,60,35.0,8.4,15.6,0.540,...,2.6,7.1,9.7,3.3,1.2,0.8,2.5,2.8,21.3,30351780
3,Ochai Agbaji,SG,22,UTA,42,5,16.4,2.0,4.2,0.461,...,0.6,1.1,1.8,0.6,0.2,0.1,0.4,1.5,5.4,3918360
4,Nickeil Alexander-Walker,SG,24,TOT,43,3,14.9,2.4,5.0,0.477,...,0.2,1.5,1.7,2.0,0.6,0.4,1.1,1.6,6.6,5009633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,Delon Wright,PG,30,WAS,35,7,23.9,2.7,5.5,0.482,...,1.2,2.2,3.4,3.9,2.0,0.3,1.0,1.3,7.2,7804878
441,Thaddeus Young,PF,34,TOR,52,9,15.1,2.1,3.8,0.551,...,1.3,1.8,3.2,1.4,1.0,0.1,0.8,1.7,4.6,8000000
442,Trae Young,PG,24,ATL,57,57,35.1,8.4,19.6,0.426,...,0.8,2.3,3.0,10.1,1.1,0.2,4.1,1.5,26.7,37096500
443,Cody Zeller,C,30,MIA,6,0,14.0,2.2,3.0,0.722,...,1.5,1.0,2.5,0.7,0.2,0.5,0.8,2.7,6.7,743922


#### Next step is to use fuzzy wuzzy to connect nicknames of players that didn't match during merge above ('https://towardsdatascience.com/fuzzywuzzy-basica-and-merging-datasets-on-names-with-different-transcriptions-e2bb6e179fbf')